In [31]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.linear_model import Lasso, ElasticNet, Ridge
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np

boston_df = pd.read_csv('./datasets/Boston.csv',index_col=0)

x = boston_df.drop(['medv'],axis=1)
y = boston_df['medv']



# method는 표준 정규 분포 변환(Standard), 최대값/최소값 정규화(MinMax), 로그변환(Log) 결정
# p_degree는 다향식 특성을 추가할 때 적용. p_degree는 2이상 부여하지 않음. 
def get_scaled_data(method='None', p_degree=None, input_data=None):
    if method == 'Standard':
        scaled_data = StandardScaler().fit_transform(input_data)
    elif method == 'MinMax':
        scaled_data = MinMaxScaler().fit_transform(input_data)
    elif method == 'Log':
        scaled_data = np.log1p(input_data)
    else:
        scaled_data = input_data

    if p_degree != None:
        scaled_data = PolynomialFeatures(degree=p_degree, 
                                         include_bias=False).fit_transform(scaled_data)
    
    return scaled_data

In [38]:
from sklearn.linear_model import ElasticNet,Ridge,Lasso
import numpy as np

def get_linear_reg_eval(model_name,params =None,x_data_n=None,y_target_n=None,verbose=True):
    coeff_df = pd.DataFrame()
    if verbose : print('######',model_name,'######')
    for param in params:
        if model_name =='Ridge' :
            model = Ridge(alpha=param)
        elif model_name =='Lasso' :
            model = Lasso(alpha=param)
        elif model_name =='ElasticNet' :
            model = ElasticNet(alpha=param,l1_ratio=0.7) 
        neg_mse_scores = cross_val_score(model,x_data_n,y_target_n,scoring='neg_mean_squared_error',cv=5)
        avg_rmse = np.mean(np.sqrt(-1*neg_mse_scores))
        print(f'alpha {param} 일 때 5폴드 세트의 평균 RMSE: {avg_rmse}')
        


In [39]:
# Ridge의 alpha값을 다르게 적용하고 다양한 데이터 변환방법에 따른 RMSE 추출. 
alphas = [0.1, 1, 10, 100]
#변환 방법은 모두 6개, 원본 그대로, 표준정규분포, 표준정규분포+다항식 특성
# 최대/최소 정규화, 최대/최소 정규화+다항식 특성, 로그변환 
scale_methods=[(None, None), ('Standard', None), ('Standard', 2), 
               ('MinMax', None), ('MinMax', 2), ('Log', None)]
for scale_method in scale_methods:
    x_data_scaled = get_scaled_data(method=scale_method[0], p_degree=scale_method[1], 
                                    input_data=x)
    print('\n## 변환 유형:{0}, Polynomial Degree:{1}'.format(scale_method[0], scale_method[1]))
    get_linear_reg_eval('Ridge', params=alphas, x_data_n=x_data_scaled, 
                        y_target_n=y, verbose=False)


## 변환 유형:None, Polynomial Degree:None
alpha 0.1 일 때 5폴드 세트의 평균 RMSE: 5.788486627032411
alpha 1 일 때 5폴드 세트의 평균 RMSE: 5.6525709656135446
alpha 10 일 때 5폴드 세트의 평균 RMSE: 5.518166280868972
alpha 100 일 때 5폴드 세트의 평균 RMSE: 5.329589628472148

## 변환 유형:Standard, Polynomial Degree:None
alpha 0.1 일 때 5폴드 세트의 평균 RMSE: 5.825992799389452
alpha 1 일 때 5폴드 세트의 평균 RMSE: 5.802889517257745
alpha 10 일 때 5폴드 세트의 평균 RMSE: 5.636831222559074
alpha 100 일 때 5폴드 세트의 평균 RMSE: 5.421374711794967

## 변환 유형:Standard, Polynomial Degree:2
alpha 0.1 일 때 5폴드 세트의 평균 RMSE: 8.827235873000435
alpha 1 일 때 5폴드 세트의 평균 RMSE: 6.871287525962447
alpha 10 일 때 5폴드 세트의 평균 RMSE: 5.4848704255333605
alpha 100 일 때 5폴드 세트의 평균 RMSE: 4.634437778261928

## 변환 유형:MinMax, Polynomial Degree:None
alpha 0.1 일 때 5폴드 세트의 평균 RMSE: 5.763570225288313
alpha 1 일 때 5폴드 세트의 평균 RMSE: 5.465045081564943
alpha 10 일 때 5폴드 세트의 평균 RMSE: 5.754163637679423
alpha 100 일 때 5폴드 세트의 평균 RMSE: 7.634919863623537

## 변환 유형:MinMax, Polynomial Degree:2
alpha 0.1 일 때 5폴드 세트의 평균 

In [49]:
import matplotlib.pyplot as  plt

st_scaler = StandardScaler()
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_breast_cancer
breast_dt = load_breast_cancer()
feature = breast_dt.data
label = breast_dt.target

feature_scale = st_scaler.fit_transform(feature)

x_train, x_test, y_train, y_test = train_test_split(feature_scale,label,test_size=0.3,random_state=0)



In [50]:
from sklearn.metrics import accuracy_score, roc_auc_score

log_reg = LogisticRegression()
log_pred =log_reg.fit(x_train,y_train).predict(x_test)

print(accuracy_score(y_test,log_pred), roc_auc_score(y_test, log_pred))


0.9766081871345029 0.9715608465608465


In [52]:
from sklearn.model_selection import GridSearchCV

params = {"penalty":['l2','l1'],'C' : [0.01, 0.1, 1,1,5,10]}

grid = GridSearchCV(log_reg, param_grid=params, scoring="accuracy", cv=3)
grid.fit(feature_scale, label)
print(grid.best_params_, grid.best_score_)

{'C': 1, 'penalty': 'l2'} 0.975392184164114


C:\Users\juno\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
18 fits failed out of a total of 36.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
18 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\juno\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\juno\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^